# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('swarm', case=False)] 
    print(len(df))
    df.to_csv('../dataset/swarm_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

45


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/swarm_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

45


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['testswarm cleanup action doesnt seem to work correctly-I have around 5 qunit '
 'tests which have timed out according to testswarm. I tried to reset those '
 'tests using cleanp action. api.php?action=cleanup It didnt work. The '
 'response from the action is {"cleanup":{"resetTimedoutRuns":0}} What can be '
 'the issue here ? ',
 'docker swarm creation fails with "remaining connection slots are '
 'reserved..."-Following the beginners docs for OSX: # Terminal started via '
 'Docker Quickstart Terminal app. $ docker-machine create -d virtualbox local '
 '$ eval "$(docker-machine env local)" $ docker run swarm create Results in '
 'pq: remaining connection slots are reserved for non-replication superuser '
 'connections. $ docker --version # Docker version 1.9.1, build a34a1d5 $ '
 'docker run swarm --version # swarm version 1.0.0 (087e245) # Virtualbox '
 'version 5.0.10 r104061 # OSX 10.11.1 (15B42) Any ideas what Im doing wrong '
 'or how the docs need to be updated? ']


# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['testswarm', 'cleanup', 'action', 'doesnt', 'seem', 'to', 'work', 'correctly', 'have', 'around', 'qunit', 'tests', 'which', 'have', 'timed', 'out', 'according', 'to', 'testswarm', 'tried', 'to', 'reset', 'those', 'tests', 'using', 'cleanp', 'action', 'api', 'php', 'action', 'cleanup', 'it', 'didnt', 'work', 'the', 'response', 'from', 'the', 'action', 'is', 'cleanup', 'what', 'can', 'be', 'the', 'issue', 'here']]


In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['action', 'seem', 'work', 'correctly', 'qunit', 'test', 'time', 'accord', 'testswarm', 'try', 'reset', 'test', 'use', 'cleanp', 'action', 'api', 'php', 'action', 'cleanup', 'work', 'response', 'action', 'cleanup', 'issue']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 4), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accord', 1),
  ('action', 4),
  ('api', 1),
  ('cleanp', 1),
  ('cleanup', 2),
  ('correctly', 1),
  ('issue', 1),
  ('php', 1),
  ('qunit', 1),
  ('reset', 1),
  ('response', 1),
  ('seem', 1),
  ('test', 2),
  ('testswarm', 1),
  ('time', 1),
  ('try', 1),
  ('use', 1),
  ('work', 2)]]

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

100%|██████████| 540/540 [09:45<00:00,  1.08s/it]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.4906048540209165

Coherence Score:  0.4146616224670476


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.059510 -0.043485       1        1  36.415799
4     -0.045655  0.050275       2        1  31.954943
3      0.023400  0.002595       3        1  12.613791
0      0.039161 -0.013544       4        1  10.654491
2      0.042605  0.004158       5        1   8.360975, topic_info=         Term        Freq       Total Category  logprob  loglift
1177  lt_stre   13.000000   13.000000  Default  30.0000  30.0000
501   country   32.000000   32.000000  Default  29.0000  29.0000
93     server   26.000000   26.000000  Default  28.0000  28.0000
23     docker  111.000000  111.000000  Default  27.0000  27.0000
68      error   41.000000   41.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
110   address    1.360662   11.724310   Topic5  -5.7033   0.3279
329      file    1.306605   16.695924   Topic5  -5.7439  -0.0661
16        use    1.238630   27.250840   Topic5  -5.7973  -0.6095
322    deploy    1.190355   24.739581   Topic5  -5.8371  -0.5526
311       add    1.116118   10.905798   Topic5  -5.9014   0.2022

[286 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1114      1  0.370408      a_viz
1114      2  0.370408      a_viz
1114      3  0.370408      a_viz
108       1  0.148532       able
108       2  0.742660       able
...     ...       ...        ...
481       1  0.165430  zookeeper
481       2  0.827152  zookeeper
653       1  0.305642         zy
653       2  0.305642         zy
653       4  0.611284         zy

[647 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 1, 3])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.023*"lt_stre" + 0.014*"error" + 0.010*"swarm" + 0.009*"manager" + '
  '0.007*"lt_key" + 0.005*"msg" + 0.005*"time_level" + 0.005*"connection" + '
  '0.004*"info" + 0.004*"join"'),
 (1,
  '0.030*"docker" + 0.025*"swarm" + 0.019*"service" + 0.016*"run" + '
  '0.011*"container" + 0.011*"network" + 0.010*"node" + 0.009*"host" + '
  '0.009*"create" + 0.009*"port"'),
 (2,
  '0.015*"server" + 0.010*"operation" + 0.008*"wflyctl" + 0.007*"service" + '
  '0.007*"swarm" + 0.007*"boot" + 0.006*"error" + 0.006*"fail" + 0.006*"cert" '
  '+ 0.005*"org"'),
 (3,
  '0.009*"cassandra" + 0.008*"docker" + 0.008*"filebeat" + 0.006*"space" + '
  '0.006*"datum" + 0.005*"server" + 0.005*"use" + 0.005*"free" + 0.005*"pool" '
  '+ 0.005*"error"'),
 (4,
  '0.033*"docker" + 0.023*"swarm" + 0.020*"country" + 0.012*"error" + '
  '0.009*"run" + 0.009*"container" + 0.009*"get" + 0.008*"server" + '
  '0.008*"image" + 0.007*"try"')]


In [12]:
num_topics = 5
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicSwarm.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  18556005  testswarm cleanup action doesn't seem to work ...   
1  34111372  docker swarm creation fails with "remaining co...   
2  34210848                    Docker-compose and Docker-swarm   
3  36642617  How to run docker-compose against docker swarm...   
4  38602903  docker swarm init could not choose an IP addre...   

                                                Body  RatingsSentiCR  \
0  I have around 5 qunit tests which have timed o...              -1   
1  Following the beginner's docs for OSX:\n\n# Te...               0   
2  I'm using docker-compose and docker swarm. I h...              -1   
3  I have managed to manually set up a docker swa...              -1   
4  Experimenting with Docker Swarm with Docker De...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         